In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [2]:
data = pd.read_csv('../data/daily_category_sales.csv', index_col=0, header=[0,1])

data = data[~(data==0).any(axis=1)]

In [3]:
categories_enc =  {'Aquatic Roots and Tubers':'1', 'Cauliflower':'2' ,'Chili Peppers':'3', 'Edible Mushrooms':'4', 'Leafy Greens':'5', 'Solanaceous Vegetables':'6'}
data.columns = ['_'.join([col[0], categories_enc[col[1]]]) for col in data.columns]
data.reset_index(inplace=True)

for i in range(1,7):
    data[f'sales_sum_{i}'] = data[f'sales_profit_sum_{i}'] + data[f'cost_sum_{i}']
    data[f'avg_sale_price_{i}'] = data[f'sales_sum_{i}'] / data[f'quantity_sum_{i}']
data['sales_sum'] = data[[f'sales_sum_{i}' for i in range(1,7)]].sum(axis=1)


data.drop(columns=[f'sales_profit_sum_{i}' for i in range(1,7)], inplace=True)
data.drop(columns=[f'avg_profit_rate_{i}' for i in range(1,7)], inplace=True)
data.drop(columns=[f'cost_sum_{i}' for i in range(1,7)], inplace=True)
data.drop(columns=[f'sales_sum_{i}' for i in range(1,7)], inplace=True)

In [4]:
data.head()

,date,quantity_sum_1,quantity_sum_2,quantity_sum_3,quantity_sum_4,quantity_sum_5,quantity_sum_6,avg_sale_price_1,avg_sale_price_2,avg_sale_price_3,avg_sale_price_4,avg_sale_price_5,avg_sale_price_6,sales_sum
0,2020-07-01,4.850,47.076,76.715,35.365,205.402,35.374,14.491505,12.716331,9.906670,10.422791,7.321202,4.998530,3478.1176
1,2020-07-02,4.600,43.943,66.064,48.510,198.362,32.199,11.548000,12.425915,10.137206,12.490806,7.106526,5.344638,3456.5429
2,2020-07-03,9.572,42.076,64.253,42.442,191.779,35.896,8.012223,11.773172,10.437872,12.871566,7.333475,5.466988,3391.6701
3,2020-07-04,5.439,55.662,81.282,47.262,236.587,57.067,21.833646,12.615177,10.167277,12.244586,7.783183,5.737169,4394.8624
4,2020-07-05,4.019,55.474,98.496,73.213,223.899,61.816,10.000000,12.582147,10.317542,10.006745,7.026290,6.654638,4471.5748


### add dummy variables of season and day_of_week

In [5]:
data['date'] = pd.to_datetime(data['date'])
data['season'] = data['date'].dt.month%12 // 3 + 1 
data['day_of_week'] = data['date'].dt.dayofweek + 1 

In [6]:
season_dummies = pd.get_dummies(data['season'])
season_dummies.columns = ['winter', 'spring', 'summer', 'fall']
season_dummies.drop(columns=['fall'], inplace=True)

dayofweek_dummies = pd.get_dummies(data['day_of_week'])
dayofweek_dummies.columns = ['_'.join(['DoW', str(i)]) for i in range(1,8)]
dayofweek_dummies.drop(columns=['DoW_7'], inplace=True)

In [7]:
data = pd.concat([data, season_dummies, dayofweek_dummies], axis=1)
data.drop(columns=['season', 'day_of_week'], inplace=True)
data.head()

,date,quantity_sum_1,quantity_sum_2,quantity_sum_3,quantity_sum_4,quantity_sum_5,quantity_sum_6,avg_sale_price_1,avg_sale_price_2,avg_sale_price_3,...,sales_sum,winter,spring,summer,DoW_1,DoW_2,DoW_3,DoW_4,DoW_5,DoW_6
0,2020-07-01,4.850,47.076,76.715,35.365,205.402,35.374,14.491505,12.716331,9.906670,...,3478.1176,0,0,1,0,0,1,0,0,0
1,2020-07-02,4.600,43.943,66.064,48.510,198.362,32.199,11.548000,12.425915,10.137206,...,3456.5429,0,0,1,0,0,0,1,0,0
2,2020-07-03,9.572,42.076,64.253,42.442,191.779,35.896,8.012223,11.773172,10.437872,...,3391.6701,0,0,1,0,0,0,0,1,0
3,2020-07-04,5.439,55.662,81.282,47.262,236.587,57.067,21.833646,12.615177,10.167277,...,4394.8624,0,0,1,0,0,0,0,0,1
4,2020-07-05,4.019,55.474,98.496,73.213,223.899,61.816,10.000000,12.582147,10.317542,...,4471.5748,0,0,1,0,0,0,0,0,0


### Double log regression

In [8]:
dummies = data[['winter', 'spring', 'summer', 'DoW_1', 'DoW_2', 'DoW_3', 'DoW_4', 'DoW_5', 'DoW_6']]
for i in range(1,7):
    y_data = data[f'quantity_sum_{i}'].copy()
    log_y = y_data.apply(lambda x: np.log(x))
    X_data = data[['avg_sale_price_1', 'avg_sale_price_2', 'avg_sale_price_3', 'avg_sale_price_4', 'avg_sale_price_5', 'avg_sale_price_6', 'sales_sum']].copy()
    log_X = X_data.apply(lambda x: np.log(x))
    
    X = pd.concat([log_X, dummies], axis=1)

    # model = sm.OLS(log_y, X)
    # result = model.fit()
    # result.summary()
    break

In [9]:
X = pd.concat([log_X, dummies], axis=1)
X

,avg_sale_price_1,avg_sale_price_2,avg_sale_price_3,avg_sale_price_4,avg_sale_price_5,avg_sale_price_6,sales_sum,winter,spring,summer,DoW_1,DoW_2,DoW_3,DoW_4,DoW_5,DoW_6
0,2.673563,2.542887,2.293208,2.343995,1.990775,1.609144,8.154247,0,0,1,0,0,1,0,0,0
1,2.446512,2.519784,2.316212,2.524993,1.961014,1.676094,8.148024,0,0,1,0,0,0,1,0,0
2,2.080968,2.465823,2.345441,2.555021,1.992449,1.698728,8.129078,0,0,1,0,0,0,0,1,0
3,3.083452,2.534901,2.319174,2.505084,2.051965,1.746966,8.388192,0,0,1,0,0,0,0,0,1
4,2.302585,2.532279,2.333846,2.303259,1.949659,1.895314,8.405496,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,2.888392,2.639057,1.774185,1.699992,1.676541,1.849920,7.318189,0,0,1,1,0,0,0,0,0
1045,2.853892,2.484907,1.840504,1.611255,1.618354,2.091666,7.486497,0,0,1,0,1,0,0,0,0
1046,2.818193,2.512203,1.802029,1.528863,1.577483,2.050538,7.530423,0,0,1,0,0,1,0,0,0
1047,2.832624,2.485242,1.831633,1.542530,1.565075,2.026922,7.695402,0,0,1,0,0,0,1,0,0


In [10]:
model = sm.OLS(log_y, X)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:         quantity_sum_1   R-squared (uncentered):                   0.980
Model:                            OLS   Adj. R-squared (uncentered):              0.980
Method:                 Least Squares   F-statistic:                              3211.
Date:                Fri, 10 Nov 2023   Prob (F-statistic):                        0.00
Time:                        23:08:40   Log-Likelihood:                         -709.27
No. Observations:                1049   AIC:                                      1451.
Df Residuals:                    1033   BIC:                                      1530.
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
avg_sale_price_1    -1.1486      0.061    -18.835      0.000      -1.268      -1.029
avg_sale_price_2    -0.2501      0.069     -3.629      0.000      -0.385      -0.115
avg_sale_price_3     0.0273      0.049      0.553      0.580      -0.069       0.124
avg_sale_price_4    -0.5430      0.064     -8.439      0.000      -0.669      -0.417
avg_sale_price_5    -0.1647      0.081     -2.029      0.043      -0.324      -0.005
avg_sale_price_6     0.3884      0.075      5.166      0.000       0.241       0.536
sales_sum            0.9024      0.028     32.587      0.000       0.848       0.957
winter              -0.0060      0.054     -0.111      0.912      -0.111       0.100
spring              -0.6160      0.052    -11.842      0.000      -0.718      -0.514
summer              -0.0148      0.052     -0.286      0.775      -0.116       0.087
DoW_1               -0.1359      0.055     -2.469      0.014      -0.244      -0.028
DoW_2               -0.0868      0.055     -1.570      0.117      -0.195       0.022
DoW_3               -0.1596      0.055     -2.891      0.004      -0.268      -0.051
DoW_4               -0.1453      0.055     -2.628      0.009      -0.254      -0.037
DoW_5               -0.0235      0.055     -0.427      0.669      -0.132       0.085
DoW_6                0.0144      0.055      0.260      0.795      -0.094       0.123
==============================================================================
Omnibus:                      303.283   Durbin-Watson:                   0.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1239.190
Skew:                          -1.317   Prob(JB):                    8.19e-270
Kurtosis:                       7.627   Cond. No.                         69.5
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""